# ***Abstract :***
# This project is built to detect the depressive based on the tweet content user posted. 

This file work on filter depressive data, because we worry about the quality of depressive tweet scrpae by the key words.

***Recommend using colab to run my program***

In [ ]:
# pip and import necessary file and library

! git clone https://github.com/Jingxuan-Bao/Tweet_Depression_Detection.git
! pip install transformers


import pandas as pd
import csv
from textblob import TextBlob
from transformers import pipeline

fatal: destination path 'Tweet_Depression_Detection' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.1 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Section 1: Load Data**

In [ ]:
# read normal tweet data from my github

normal_tweet = pd.read_csv('/content/Tweet_Depression_Detection/dataset/normal_tweet.csv')
print(normal_tweet.head())

                                                text  label
0  in class. im EARLY. bummed RYAN SEACREST never...      0
1                                   i wanna go home       0
2                                  Our AC is broken       0
3  feels that the only place where Federer (&amp;...      0
4  @tommcfly Yeah tom, you are always the one tha...      0


In [ ]:
normal_tweet

,text,label
0,in class. im EARLY. bummed RYAN SEACREST never...,0
1,i wanna go home,0
2,Our AC is broken,0
3,feels that the only place where Federer (&amp;...,0
4,"@tommcfly Yeah tom, you are always the one tha...",0
...,...,...
59995,It's 8:40am... I've been awake for nearly 2 ho...,0
59996,ughh mom`s making me cook dinner when i`m in t...,0
59997,"argh. netregistry is pissing me off today, pos...",0
59998,@gemmak500 It's amazing how many couples you s...,0


In [ ]:
# read depressive tweet data from my github

depressive_tweet = pd.read_csv('/content/Tweet_Depression_Detection/dataset/depressive_tweets.csv')
print(depressive_tweet.head())

   Unnamed: 0             tweet.id           created_at  \
0           0  1447537898572574730  2021-10-11 12:21:43   
1           1  1447540582490988553  2021-10-11 12:32:23   
2           2  1447807717859491842  2021-10-12 06:13:53   
3           3  1448076026219692033  2021-10-13 00:00:03   
4           4  1448382047375040513  2021-10-13 20:16:04   

                                                text          location  \
0  Open discussion. Between the Transfer Portal a...  Cheyenne Wyoming   
1  Plenty of things are changing in my life and t...               NaN   
2  I feel a little hopeless. Anyone else? #hopele...               NaN   
3  Which is more healthy? Hope, or hopelessness? ...        Denver, CO   
4  So someone tell me how do I get over #HOPELESS...    Portland Or .    

   retweet  favorite  
0        0         0  
1        0         0  
2        0         0  
3        0         0  
4        0         2  


In [ ]:
depressive_tweet

,Unnamed: 0,tweet.id,created_at,text,location,retweet,favorite
0,0,1447537898572574730,2021-10-11 12:21:43,Open discussion. Between the Transfer Portal a...,Cheyenne Wyoming,0,0
1,1,1447540582490988553,2021-10-11 12:32:23,Plenty of things are changing in my life and t...,NaN,0,0
2,2,1447807717859491842,2021-10-12 06:13:53,I feel a little hopeless. Anyone else? #hopele...,NaN,0,0
3,3,1448076026219692033,2021-10-13 00:00:03,"Which is more healthy? Hope, or hopelessness? ...","Denver, CO",0,0
4,4,1448382047375040513,2021-10-13 20:16:04,So someone tell me how do I get over #HOPELESS...,Portland Or .,0,2
...,...,...,...,...,...,...,...
24142,24142,1459521498842992642,2021-11-13 14:00:16,Just got banned from a server F #sad,Jakarta Capital Region,0,1
24143,24143,1459521611997003777,2021-11-13 14:00:43,I literally cried during my exam and the cam i...,بيت أمك,0,0
24144,24144,1459524263946326017,2021-11-13 14:11:15,No one can be happy with a guy like me. That's...,"Varanasi, Uttar Pradesh, India",0,0
24145,24145,1459530315437785095,2021-11-13 14:35:18,arrived at my house but Am I Home? #deep #sad ...,they19sea,1,3


# **Section 2: Filter Depressive Tweets through sentiment analysis and pre-trained model**

In [ ]:
# preprocess the depressive tweet, extract the target content, clean null data

depressive_tweet['label'] = 1
depressive_tweet = depressive_tweet[['text', 'label']]
depressive_tweet = depressive_tweet.dropna()

In [ ]:
depressive_tweet

,text,label
0,Open discussion. Between the Transfer Portal a...,1
1,Plenty of things are changing in my life and t...,1
2,I feel a little hopeless. Anyone else? #hopele...,1
3,"Which is more healthy? Hope, or hopelessness? ...",1
4,So someone tell me how do I get over #HOPELESS...,1
...,...,...
24142,Just got banned from a server F #sad,1
24143,I literally cried during my exam and the cam i...,1
24144,No one can be happy with a guy like me. That's...,1
24145,arrived at my house but Am I Home? #deep #sad ...,1


Filter Depressive Tweet through sentiment analysis by TextBlob, we only keep the tweet with less than -0.2 sentiment score.

In [ ]:
depressive_tweets = depressive_tweet['text']
print(depressive_tweets)
max_tweets = len(depressive_tweets)
print(max_tweets)


negative_tweets = []
for tweet in depressive_tweets:
    sentiment = TextBlob(tweet).sentiment
    if sentiment.polarity < -0.2:
        negative_tweets.append((tweet, sentiment))


percentage_negative = len(negative_tweets) / max_tweets * 100
print(f"Percentage of negative tweets: {percentage_negative}%")


0        Open discussion. Between the Transfer Portal a...
1        Plenty of things are changing in my life and t...
2        I feel a little hopeless. Anyone else? #hopele...
3        Which is more healthy? Hope, or hopelessness? ...
4        So someone tell me how do I get over #HOPELESS...
                               ...                        
24142                 Just got banned from a server F #sad
24143    I literally cried during my exam and the cam i...
24144    No one can be happy with a guy like me. That's...
24145    arrived at my house but Am I Home? #deep #sad ...
24146    Being spoken down to rn at @starbucks and reme...
Name: text, Length: 24147, dtype: object
24147
Percentage of negative tweets: 19.84511533523833%


In [ ]:
print(len(negative_tweets))

4792


Filter the depressive by pre-trained model - Facebook bart large mnli

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

labels = ["depressive", "non-depressive"]

index = 0

# Classify tweets
classified_tweets = []
for tweet in negative_tweets:
    result = classifier(tweet, labels)
    print(result)
    print("I am working on num : ", index)
    index += 1
    label = result["labels"][0]
    if label == "depressive":
        classified_tweets.append(tweet)

depressive_tweets = classified_tweets

print("Depressive tweets percentage by bart-large-mnli model :")
print(len(classified_tweets) / len(negative_tweets) * 100)

Streaming output truncated to the last 5000 lines.
{'sequence': 'Depression is this condition where we put this invisible cloak around us. Giving us the opportunity to feel save for once in our life. Yet is makes us feel even worse than ever before. #Islabot #Depression', 'labels': ['depressive', 'non-depressive'], 'scores': [0.980010986328125, 0.01998909004032612]}
I am working on num :  2293
{'sequence': 'Late night overthinking #depression #anxiety #lowmood', 'labels': ['depressive', 'non-depressive'], 'scores': [0.9955814480781555, 0.0044185565784573555]}
I am working on num :  2294
{'sequence': 'Why did it have to end like this??? Fucking pain!!! #SFGiants #depression', 'labels': ['depressive', 'non-depressive'], 'scores': [0.9974989295005798, 0.00250111217610538]}
I am working on num :  2295
{'sequence': "I  hate everyone here. I'll kill myself . #depression @Imheret45140132", 'labels': ['depressive', 'non-depressive'], 'scores': [0.9939321875572205, 0.006067854352295399]}
I am w

In [ ]:
classified_tweets

[('“Our world today so desperately hungers for hope, yet uncounted people have almost given up. There is despair and hopelessness on every hand. Let us be faithful in proclaiming the hope that is in Jesus.” \n\nBilly Graham\n#hope #hopelessness #faithful #Jesus',
  Sentiment(polarity=-0.6, subjectivity=1.0)),
 ('“Go if you have to, but remember, don’t come back if you fail, which you will.”\n\nDid you hear this sentiment from anyone in your life or career?\n\n#canidoit #faith #hopelessness #feelingletdown #willdoit #unknowpath #risk',
  Sentiment(polarity=-0.25, subjectivity=0.14999999999999997)),
 ('#Artists without an air of #loneliness , are #boring !',
  Sentiment(polarity=-1.0, subjectivity=1.0)),
 ('Getting a bit tired of losing my mind \n\n#bipolar #depression #loneliness #anxiety\n\nSocially incompetent',
  Sentiment(polarity=-0.375, subjectivity=0.5333333333333333)),
 ('Maturity is when you realize\nYour image, relations with people and people themselves are useless\nALL you w

In [ ]:
with open('/content/tweets.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Tweet'])
    for tweet in classified_tweets:
        writer.writerow([tweet[0]])